In [ ]:
import math
import numpy as np
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os


GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS839_project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['anmol_predictions', 'schema_linking.ipynb', 'Dataset']


In [ ]:
import json

schema_path = os.path.join(GOOGLE_DRIVE_PATH,'Dataset/tables.json')
# data_path = os.path.join(GOOGLE_DRIVE_PATH,'Dataset/train_spider.json')

def get_entities(schema_path):
    with open(table_path, 'r', encoding='utf8') as f:
        table_datas = json.load(f)

    output_tab = {}
    tables = {}
    tabel_name = set()
    for i in range(len(table_datas)):
        table = table_datas[i]
        temp = {}
        temp['col_map'] = table['column_names']
        temp['table_names'] = table['table_names']
        tmp_col = []
        for cc in [x[1] for x in table['column_names_original']]:
            if cc not in tmp_col:
                tmp_col.append(cc)
        table['col_set'] = tmp_col
        db_name = table['db_id']
        tabel_name.add(db_name)
        table['schema_content'] = [col[1] for col in table['column_names']]
        table['col_table'] = [col[0] for col in table['column_names']]
        output_tab[db_name] = temp
        tables[db_name] = table

    return tables

db_dict = get_entities(table_path)

# Use 'col_set' from db_dict[<db_id>'] to match column names
# Use 'table_names' from db_dict[<db_id>] to match table names

In [ ]:
# db_id = 'car_1'
# db_dict[db_id]['col_set']
# db_dict[db_id]['table_names']

In [ ]:
import difflib

def get_matches(entity_lst, known_lst):
  match_dict = {}
  for entity in entity_lst:
    match_1 = difflib.get_close_matches(entity, known_lst, n=1, cutoff=0.3)
    if match_1 != []:
      match_dict[entity] = match_1[0]
    else:
      entity_lst.remove(entity)
  return entity_lst, match_dict

def replace_entity(pred, entity_lst, match_dict):
  for entity in entity_lst:
    pred = pred.replace(entity, match_dict[entity])
  return pred

In [ ]:
# pred = 'SELECT count(*) FROM airlines as from abc join from xyz'.lower()
# pred = 'SELECT count(number), sum(abc), T.ncv, A1.xyz, grade FROM High_School AS T1'.lower()

In [ ]:
# Table correction
def correct_tables(db_dict, db_id, pred):
  out = re.findall(r'from (.*?) (as|join|$)', pred)
  tables = [match[0] for match in out]

  out = re.findall(r'join (.*?) (where|groupby|having|$)', pred)
  table_2 =  [match[0] for match in out]
  tables = tables + table_2

  tables, match_dict = get_matches(tables, db_dict[db_id]['table_names_original'])
  pred = replace_entity(pred, tables, match_dict)

  return pred

# Column correction
def correct_columns(db_dict, db_id, pred):
  columns = re.findall(r'select (.*?) from', pred)
  if columns != []:
    columns = columns[0]
    columns = columns.split(',')

    agg_fn_lst = ['count','sum','avg','min','max']
    for agg_fn in agg_fn_lst:
      columns = [term.replace(agg_fn+'(','') for term in columns]
      columns = [term.replace(')','') for term in columns]

    columns = [re.sub(r'.*\.', '', term) for term in columns]

    columns = [term.strip() for term in columns]
    if '*' in columns:
      columns.remove('*')
    # print(columns)
    columns, match_dict = get_matches(columns, db_dict[db_id]['col_set'])
    # print(columns)
    pred = replace_entity(pred, columns, match_dict)

  return pred

In [ ]:
row = results.iloc[812]
print(i)
db_id = row['DB']
gold = row['Gold']
pred = row['Pred'].lower()
print(db_id)
print(pred)
pred = correct_tables(db_dict, db_id, pred)
pred = correct_columns(db_dict, db_id, pred)
print(pred)

812
world_1
select name, year_of_independence, surface_area from
select name, LifeExpectancy, SurfaceArea from


In [ ]:
results = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH,'anmol_predictions/Results.csv'))
row = results.iloc[0]
i = 0
preds_ec = []
for idx, row in results.iterrows():
  print(i)
  db_id = row['DB']
  gold = row['Gold']
  pred = row['Pred'].lower()
  print(db_id)
  print(pred)
  try:
    pred = correct_tables(db_dict, db_id, pred)
    pred = correct_columns(db_dict, db_id, pred)
  except:
    pass
  print(pred)
  preds_ec.append(pred)
  # if i==10:
  #   break
  i+=1

0
network_1
select t1.name, count(*) from high_schoolers as t
select t1.name, count(*) from Highschooler as t
1
cre_Doc_Template_Mgt
select template_type_code from templates group by template_type_
select Template_Type_Code from templates group by template_type_
2
dog_kennels
select breed_name from dogs order by count(*) desc li
select breed_name from dogs order by count(*) desc li
3
dog_kennels
select charge_amount from charges order by charge_amount desc
select charge_amount from charges order by charge_amount desc
4
employee_hire_evaluation
select count(*), name from shop</s>
select count(*), Name from shop</s>
5
flight_2
select count(*) from airlines where country = "usa"</s>
select count(*) from airlines where country = "usa"</s>
6
dog_kennels
select owner_id, last_name from treatments where spending_
select owner_id, last_name from treatments where spending_
7
network_1
select t1.name from likes as t1 join student as t2
select t1.name from Likes as t1 join student as t2
8
singer


In [ ]:
# results['Preds_ec'] = preds_ec
# results.to_csv(os.path.join(GOOGLE_DRIVE_PATH,'anmol_predictions/Results_ec.csv'))

In [ ]:
# !pip install -q -U datasets > /dev/null
# # !pip install transformers
# !pip install --no-cache-dir transformers sentencepiece
# !pip3 install sqlparse nltk

# from transformers import AutoModelWithLMHead, AutoTokenizer
# from datasets import load_dataset
# import random, warnings
# import nltk
# warnings.filterwarnings("ignore")

In [ ]:
# valid_dataset = load_dataset('spider', split='validation')

['number', ' abc', 'ncv', 'xyz', ' grade']

In [ ]:
# i = 100
# print(valid_dataset[i]['db_id'])
# print(valid_dataset[i]['question'])
# print(valid_dataset[i]['query'])

['number', ' abc', ' t.ncv', ' a1.xyz', ' grade']